In [6]:
#r "BoSSSpad.dll"
#r "XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Databases loaded: 
Capacity: 0
Count: 0



Error: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in C:\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 217
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in C:\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 104
   at Submission#7.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

init workflow


In [7]:
BoSSSshell.WorkflowMgm.Init("XESFvsCNS_WedgeFlow");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


Project name is set to 'XESFvsCNS_WedgeFlow'.
Default Execution queue is chosen for the database.
Opening existing database '\\dc3\userspace\sebastian\cluster\XESFvsCNS_WedgeFlow'.


{ Session Count = 17; Grid Count = 17; Path = \\dc3\userspace\sebastian\cluster\XESFvsCNS_WedgeFlow }

In [8]:
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var dt = database.Sessions.Last().CreationTime;
dt


2024-03-23 12:15:47Z

In [9]:
var sessions = database.Sessions;
sessions

#0: IBMWedgeFlow	IBMWedgeFlow_p1_xCells60_yCells40_isUniform_True_ref10*	4/17/2024 4:51:29 PM	34750bbf...
#1: IBMWedgeFlow	IBMWedgeFlow_p1_xCells120_yCells80_isUniform_True_ref10*	4/17/2024 3:22:20 PM	5418877e...
#2: IBMWedgeFlow	IBMWedgeFlow_p1_xCells15_yCells10_CFLFrac0.1_RK1_s0=1.0E-03_lambdaMax_agg0.3_RESTART12	3/23/2024 2:14:34 PM	a4ae7688...
#3: IBMWedgeFlow	IBMWedgeFlow_p1_xCells15_yCells10_isUniform_False_ref10*	3/24/2024 10:24:29 AM	c5c39c1c...
#4: IBMWedgeFlow	IBMWedgeFlow_p1_xCells15_yCells10_isUniform_False_ref6*	3/24/2024 10:21:16 AM	97aa26e9...
#5: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/23/2024 3:31:59 PM	0edcfdf3...
#6: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	3/23/2024 3:32:30 PM	d9b84916...
#7: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/23/2024 3:26:15 PM	df3ffe04...
#8: IBMWedgeFlow	IBMWedgeFlow_p1_xCells15_yCells10_CFLFrac0.1_RK1_s0=1.0E-03_lambdaMax_agg0.3_RESTART12*	3/23/2024 3:

In [10]:
MatrixAssembler aaa = new MatrixAssembler(12);
var si = database.Sessions.Pick(12); // pick a session to compare
var ti = si.Timesteps;
si.ID

0b3708f4-9068-4d2e-bc3b-4f328bde97c5

In [12]:
//si.Export().Do() //export the plot

In [13]:
var flds = si.Timesteps.Last().Fields;
var tp= new Tecplot(flds.Last().GridDat,2);
tp.PlotFields(si.Name,0,flds);


## Enthalpy error
enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(\gamma -1)(\rho E - \rho e_{kin}),~\rho e_{kin}=\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$

For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

In the case of the WedgeFlow ($p_\text{in}=1,\rho_\text{in}=1$) presented, the v-velocity is zero and we have
$$ h_{in} = \rho E_{in}+1$$
We prescribe the inflow conditions by the Mach number $M_{in}=2$. From those the Energy is computed as 
$$ \rho E_{in} = (\rho e_{inner})_\text{in}+(\rho e_{kin})_\text{in}=\frac{p_\text{in}}{\gamma-1}+\frac{\rho_\text{in}}{2}u_\text{in}^2= \frac{1}{\gamma-1}+\frac{1}{2}u_{in}^2$$ 
where $u_{in} = M_{in}\sqrt{\gamma}$. In total we obtain:
$$ h_\text{in}=\rho E_{in}+1=\frac{\gamma}{\gamma-1}+\frac{1}{2}u_{in}^2$$


In [14]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = si.Timesteps.Last().GetField("h");
var enthalpy_exact = (DGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 2.0;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

6.300000000000001

Compute the enthalpy error

In [15]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2Norm();
var EE = enthalpy_end.L2Error(enthalpy_exact)/EEN;
EE

0.24152460611032395

In [18]:
// pick a session to compare
var si = database.Sessions.Pick(0); // pick a session to compare
si.Timesteps

#0:  { Time-step: 0; Physical time: 0s; Fields: rho, m0, m1, rhoE, levelSetGradient0, levelSetGradient1, artificialViscosity, u0, u1, p, h, S, Ma, mpiRank, sensor, cfl, levelSet, MPIrank; Name:  }
#1:  { Time-step: 50000; Physical time: 0.17726192749367173s; Fields: rho, m0, m1, rhoE, levelSetGradient0, levelSetGradient1, artificialViscosity, u0, u1, p, h, S, Ma, mpiRank, sensor, cfl, levelSet, MPIrank; Name:  }
#2:  { Time-step: 100000; Physical time: 0.41412515432495767s; Fields: rho, m0, m1, rhoE, levelSetGradient0, levelSetGradient1, artificialViscosity, u0, u1, p, h, S, Ma, mpiRank, sensor, cfl, levelSet, MPIrank; Name:  }
#3:  { Time-step: 150000; Physical time: 0.6511714807931442s; Fields: rho, m0, m1, rhoE, levelSetGradient0, levelSetGradient1, artificialViscosity, u0, u1, p, h, S, Ma, mpiRank, sensor, cfl, levelSet, MPIrank; Name:  }
#4:  { Time-step: 200000; Physical time: 0.8886786746058578s; Fields: rho, m0, m1, rhoE, levelSetGradient0, levelSetGradient1, artificialViscosit

plot the enthalpy error for all saved timesteps of the CNS simulations

In [23]:
var enthalpyerrors= new List<double>();
var times= new List<double>();
foreach(var ts in si.Timesteps){
    var enthalpy= ts.GetField("h");
    enthalpy_exact = (DGField) enthalpy.Clone();
    enthalpy_exact.Clear();
    enthalpy_exact.ProjectField(1.0,t => enthalpy_inflow);
    enthalpy_exact.Identification= "h_exact";
    enthalpyerrors.Add(enthalpy.L2Error(enthalpy_exact)/EEN);
    times.Add(ts.PhysicalTime);
}
Plot(times,enthalpyerrors,"enthalpyerrorCNS")

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.2207 
 
 
 
 
 0.2208 
 
 
 
 
 0.2209 
 
 
 
 
 0.221 
 
 
 
 
 0.2211 
 
 
 
 
 0.2212 
 
 
 
 
 0.2213 
 
 
 
 
 0.2214 
 
 
 
 
 0.2215 
 
 
 
 
 0.2216 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 
 
 
 
 enthalpyerrorCNS 
 
 
 enthalpyerrorCNS